In [1]:
import subprocess
import sys
import os

# Define a function to install packages
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of packages to install
packages = [
    "pandas",
    "numpy",
    "scikit-learn",
    "pickle-mixin",
    "pyarrow"
]

# Install each package
for package in packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

#assign correct directory
name = sys.platform

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVC
import random as rm
from sklearn.metrics import classification_report, f1_score
import pickle as pk
from sklearn.preprocessing import PolynomialFeatures
from itertools import combinations
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Lasso
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import mahalanobis

In [9]:
import pandas as pd

# 1. Load the Stata dataset
df = pd.read_stata("CMF/CMF_1860.dta")

# 2. Convert "materials_value*" and "production_values*" columns to numeric (force conversion)
# Identify columns whose names start with either pattern and convert them (errors='coerce' converts non-numeric values to NaN)
for col in df.columns:
    if col.startswith("materials_value") or col.startswith("production_values"):
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. Generate aggregated features for "materials_value" and "production_values"
# For each base variable, sum all columns whose names start with that variable.
for base in ['materials_value', 'production_values']:
    matching_cols = [col for col in df.columns if col.startswith(base)]
    # Sum across the matching columns. The skipna=True means only non-missing numbers are summed.
    df[base] = df[matching_cols].sum(axis=1, skipna=True)
    # In case all values were missing, replace NaN with 0.
    df[base] = df[base].fillna(0)

# 4. Rename the aggregated 'production_values' column to 'output'
df.rename(columns={'production_values': 'output'}, inplace=True)

# 5. Keep only specific columns.
#    The Stata "keep" command retains:
#    file_name, firm_number, firm_name, output, capital, materials_value, hands_female,
#    hands_male, avg_wage_female, avg_wage_male, all variables starting with "ind_",
#    materials_value1-5, and production_values1-5.
# First build a list of columns to keep:
base_keep = [
    'file_name', 'firm_number', 'firm_name', 'output', 'capital',
    'materials_value', 'hands_female', 'hands_male', 'avg_wage_female', 'avg_wage_male'
]
# Add any columns starting with "ind_"
ind_cols = [col for col in df.columns if col.startswith("ind_")]
# Explicitly add materials_value1 through materials_value5 if they exist
mv_cols = [f"materials_value{i}" for i in range(1, 6) if f"materials_value{i}" in df.columns]
# Explicitly add production_values1 through production_values5 if they exist
# (These will later be renamed)
pv_cols = [f"production_values{i}" for i in range(1, 6) if f"production_values{i}" in df.columns]

# Retain only these columns (if they exist in the dataframe)
cols_to_keep = base_keep + ind_cols + mv_cols + pv_cols
df = df[cols_to_keep]

# 6. For specified numeric columns, ensure they are numeric and replace missing values with 0.
#    The variables are: output, capital, materials_value, hands_female, hands_male,
#    avg_wage_female, and avg_wage_male.
for col in ['output', 'capital', 'materials_value', 'hands_female', 'hands_male', 'avg_wage_female', 'avg_wage_male']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# 7. Rename production values variables
#    In Stata, the loop renames each variable starting with "production_values" (other than the aggregated one)
#    to output1, output2, etc.
# Find all columns that start with "production_values". (These are the individual ones kept earlier.)
pv_individual = [col for col in df.columns if col.startswith("production_values")]
for i, col in enumerate(pv_individual, start=1):
    df.rename(columns={col: f"output{i}"}, inplace=True)

# 8. Reorder columns so that a specific set of variables come first.
#    The desired order (moved to front) is:
#    file_name, firm_number, firm_name, capital, materials_value, hands_female,
#    hands_male, avg_wage_female, avg_wage_male, output.
#    Other columns will follow in their original order.
front_cols = ['file_name', 'firm_number', 'firm_name', 'capital', 'materials_value',
              'hands_female', 'hands_male', 'avg_wage_female', 'avg_wage_male', 'output']
remaining_cols = [col for col in df.columns if col not in front_cols]
df = df[front_cols + remaining_cols]

# 9. Save the resulting dataframe to a Stata .dta file.
output_file = r"CMF_to_predict/1860_to_predict.dta"
df.to_stata(output_file, write_index=False)

print("Data processing complete. Output saved to:", output_file)

475
Index(['ind_detailed', 'ind_broadest', 'ind_leontief', 'ind_granular',
       'file_name', 'firm_number', 'industry_raw', 'industry_freq', 'fips',
       'state',
       ...
       'machine_category12', 'machine_unit13', 'machine_kind13',
       'machine_category13', 'machine_unit14', 'machine_kind14',
       'machine_category14', 'machine_unit15', 'machine_kind15',
       'machine_category15'],
      dtype='object', length=475)


/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_42261/3190240245.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[base] = df[matching_cols].sum(axis=1, skipna=True)
/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_42261/3190240245.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[base] = df[matching_cols].sum(axis=1, skipna=True)


Data processing complete. Output saved to: CMF_to_predict/1860_to_predict.dta


In [3]:
def ratio(df, var1, var2):
    df[f'{var1}&{var2}'] = df[var1]/df[var2]

In [4]:
print(df.columns)

Index(['file_name', 'firm_number', 'firm_name', 'capital', 'materials_value',
       'hands_female', 'hands_male', 'avg_wage_female', 'avg_wage_male',
       'output', 'ind_detailed', 'ind_broadest', 'ind_leontief',
       'ind_granular', 'materials_value1', 'materials_value2',
       'materials_value3', 'materials_value4', 'materials_value5', 'output1',
       'output2', 'output3', 'output4', 'output5'],
      dtype='object')


In [9]:
# Read data
input_1860 = pd.read_stata("CMF_to_predict/1860_to_predict.dta")
# get the columns
input_1860.columns

Index(['file_name', 'firm_number', 'firm_name', 'capital', 'materials_value',
       'hands_female', 'hands_male', 'avg_wage_female', 'avg_wage_male',
       'output', 'ind_detailed', 'ind_broadest', 'ind_leontief',
       'ind_granular', 'materials_value1', 'materials_value2',
       'materials_value3', 'materials_value4', 'materials_value5', 'output1',
       'output2', 'output3', 'output4', 'output5'],
      dtype='object')

In [10]:
#create specific ratios
ratio(input_1860, 'materials_value', 'output')
ratio(input_1860, 'materials_value', 'capital')
ratio(input_1860, 'capital', 'output')
ratio(input_1860, 'capital', 'materials_value')
ratio(input_1860, 'output', 'materials_value')
ratio(input_1860, 'output', 'capital')
ratio(input_1860, 'hands_male', 'avg_wage_male')
ratio(input_1860, 'hands_female', 'avg_wage_female')
ratio(input_1860, 'avg_wage_male', 'output')
ratio(input_1860, 'avg_wage_female', 'output')

var_list = ["capital", "materials_value", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output", 'materials_value&output', 'materials_value&capital', 'capital&output', 'capital&materials_value', 'output&materials_value', 'output&capital','hands_male&avg_wage_male', 'hands_female&avg_wage_female', 'avg_wage_male&output', 'avg_wage_female&output' ]

input_1860.replace({np.inf: 0, -np.inf: 0, np.nan: 0}, inplace=True)

In [11]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis

industry_list = input_1860["ind_broadest"].unique().tolist()
final_df = []

for industry in industry_list:
    input_df = input_1860[input_1860["ind_broadest"] == industry].copy()  # Ensure a copy

    if input_df.empty:
        continue  # Skip if there's no data for this industry

    # Compute mean and covariance matrix
    mean_vec = np.mean(input_df[var_list], axis=0)
    cov_matrix = np.cov(input_df[var_list], rowvar=False)

    # Check if covariance matrix is singular
    if np.linalg.det(cov_matrix) == 0:
        print(f"Skipping {industry}: Covariance matrix is singular")
        continue

    inv_cov_matrix = np.linalg.inv(cov_matrix)

    # Compute Mahalanobis distance for each firm
    distances = np.array([mahalanobis(row, mean_vec, inv_cov_matrix) for row in input_df[var_list].to_numpy()])
    
    # Compute contributions
    contributions = np.array([(row - mean_vec) * np.dot(inv_cov_matrix, (row - mean_vec)) for row in input_df[var_list].to_numpy()])
    contributions_df = pd.DataFrame(contributions, columns=[col + "_con" for col in var_list])

    # Identify the most anomalous variable for each row
    input_df['most_anomalous_var'] = contributions_df.idxmax(axis=1)
    input_df['mahalanobis'] = distances

    # Combine input_df with contributions_df (ensuring correct alignment)
    input_df = pd.concat([input_df.reset_index(drop=True), contributions_df.reset_index(drop=True)], axis=1)

    final_df.append(input_df)  # Append a COPY

# Combine all industry DataFrames into one
final_df = pd.concat(final_df, ignore_index=True)

In [12]:
final_df['dist_squared'] = final_df['mahalanobis'] ** 2
for var in var_list:
    final_df[var + "_divided"] = final_df[var+"_con"]/final_df['dist_squared']
final_df['total'] = 0
for var in var_list:
    final_df['total'] = final_df['total'] + final_df[var + "_divided"]
final_df

,file_name,firm_number,firm_name,capital,materials_value,hands_female,hands_male,avg_wage_female,avg_wage_male,output,...,materials_value&capital_divided,capital&output_divided,capital&materials_value_divided,output&materials_value_divided,output&capital_divided,hands_male&avg_wage_male_divided,hands_female&avg_wage_female_divided,avg_wage_male&output_divided,avg_wage_female&output_divided,total
0,AL_6_autaga_00003_r_L.jpg,1,A.C. Baker,1000.0,175.0,0.0,4.0,0.0,50.0,1200.0,...,0.087781,0.017929,0.101521,-0.053000,-0.026855,0.037303,0.042559,0.065348,0.008409,1.0
1,AL_6_autaga_00003_r_L.jpg,2,J.C. Zeigler,3000.0,650.0,0.0,5.0,0.0,75.0,2800.0,...,0.063085,0.002210,0.076563,-0.016749,0.013578,-0.018721,0.056924,-0.012637,0.003225,1.0
2,AL_6_autaga_00003_r_L.jpg,3,William Spears,4300.0,1000.0,0.0,5.0,0.0,100.0,2500.0,...,0.074867,-0.001572,0.108809,0.059462,0.059093,0.105717,0.064572,0.039275,0.007664,1.0
3,AL_6_autaga_00003_r_L.jpg,7,James B Nixon,1000.0,1000.0,0.0,2.0,0.0,25.0,2500.0,...,-0.000338,-0.000448,0.180133,-0.046305,0.005507,0.142660,0.032313,0.113484,-0.003246,1.0
4,AL_6_autaga_00003_r_L.jpg,8,C C Thompson,1000.0,1100.0,0.0,5.0,0.0,100.0,3600.0,...,-0.001336,0.026890,0.190267,-0.016249,0.158963,0.065992,0.041296,-0.007564,0.000487,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121525,dc_6_dist columbia_00120_r_L.jpg,3,Geo Small,1000.0,2854.0,0.0,2.0,0.0,60.0,5000.0,...,0.056918,0.191309,-0.012567,0.184273,-0.006406,0.021771,0.173345,0.320921,-0.057158,1.0
121526,dc_6_dist columbia_00120_r_L.jpg,5,Jno Wagner,3000.0,4850.0,0.0,2.0,0.0,46.0,7000.0,...,-0.002507,-0.008955,0.040813,0.121318,0.039659,0.000997,0.161787,0.538318,-0.063076,1.0
121527,dc_6_dist columbia_00123_r_L.jpg,3,K Koffin,1000.0,551.0,0.0,3.0,0.0,105.0,1125.0,...,0.003268,-0.000291,0.004652,0.030232,0.010487,0.009444,-0.016995,0.893621,0.064132,1.0
121528,dc_6_dist columbia_00124_r_L.jpg,7,M. W. Galt & Bro,125000.0,21300.0,0.0,15.0,0.0,600.0,44600.0,...,-0.003148,0.129358,-0.019527,-0.007380,-0.003557,0.001382,0.004013,0.007249,-0.000351,1.0


In [1]:
#output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value", "hands_female", "hands_male", "tot_wages", "output"]

for var in var_list:
    batch = []
    for ind in industry_list:
        ind_specific = final_df[final_df["ind_broadest"] == ind].copy()  # Ensure a copy
        #ORIGINAL CUTOFF : .0001
        quantile_low = ind_specific[var + '_divided'].quantile(0.0001)
        quantile_high = ind_specific[var + '_divided'].quantile(1 - 0.0001)

        # Filter the lowest and highest 0.01% values
        lowest_values = ind_specific[ind_specific[var + '_divided'] <= quantile_low]
        highest_values = ind_specific[ind_specific[var + '_divided'] >= quantile_high]

        # Combine results
        result = pd.concat([lowest_values, highest_values])

        batch.append(result)

    batch = pd.concat(batch, ignore_index=True)

    if "materials_value&output" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5", "hands_female", "hands_male", "tot_wages", "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        variable_check['correct_materials_value_1'] = None
        variable_check['correct_materials_value_2'] = None
        variable_check['correct_materials_value_3'] = None
        variable_check['correct_materials_value_4'] = None
        variable_check['correct_materials_value_5'] = None
        variable_check['correct_output_1'] = None
        variable_check['correct_output_2'] = None
        variable_check['correct_output_3'] = None
        variable_check['correct_output_4'] = None
        variable_check['correct_output_5'] = None
        variable_check.to_csv(f"check/1870/{var}_check_1870.csv", index=False)
    elif "output&materials_value" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5", "hands_female", "hands_male", "tot_wages", "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        variable_check['correct_output_1'] = None
        variable_check['correct_output_2'] = None
        variable_check['correct_output_3'] = None
        variable_check['correct_output_4'] = None
        variable_check['correct_output_5'] = None
        variable_check['correct_materials_value_1'] = None
        variable_check['correct_materials_value_2'] = None
        variable_check['correct_materials_value_3'] = None
        variable_check['correct_materials_value_4'] = None
        variable_check['correct_materials_value_5'] = None
        variable_check.to_csv(f"check/1860/{var}_check_1860.csv", index=False)

    elif "materials_value" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        parts = var.split("&")
        if len(parts) == 2:
            left, right=parts
            if "materials_value" in left:
                variable_check['correct_materials_value_1'] = None
                variable_check['correct_materials_value_2'] = None
                variable_check['correct_materials_value_3'] = None
                variable_check['correct_materials_value_4'] = None
                variable_check['correct_materials_value_5'] = None
                variable_check[f'correct_{right}'] = None
            else:
                variable_check[f'correct_{left}'] = None
                variable_check['correct_materials_value_1'] = None
                variable_check['correct_materials_value_2'] = None
                variable_check['correct_materials_value_3'] = None
                variable_check['correct_materials_value_4'] = None
                variable_check['correct_materials_value_5'] = None
        else:
            variable_check['correct_materials_value_1'] = None
            variable_check['correct_materials_value_2'] = None
            variable_check['correct_materials_value_3'] = None
            variable_check['correct_materials_value_4'] = None
            variable_check['correct_materials_value_5'] = None
        variable_check.to_csv(f"check/1860/{var}_check_1860.csv", index=False)

    elif "output" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        parts = var.split("&")
        if len(parts) == 2:
            left, right=parts
            if "output" in left:
                variable_check['correct_output_value_1'] = None
                variable_check['correct_output_value_2']= None
                variable_check['correct_output_value_3'] = None
                variable_check['correct_output_value_4'] = None
                variable_check['correct_output_value_5'] = None
                variable_check[f'correct_{right}'] = None
            else:
                variable_check[f'correct_{left}'] = None
                variable_check['correct_output_value_1'] = None
                variable_check['correct_output_value_2'] = None
                variable_check['correct_output_value_3'] = None
                variable_check['correct_output_value_4'] = None
                variable_check['correct_output_value_5'] = None
        else:
            variable_check['correct_output_value_1'] = None
            variable_check['correct_output_value_2'] = None
            variable_check['correct_output_value_3'] = None
            variable_check['correct_output_value_4'] = None
            variable_check['correct_output_value_5'] = None
        variable_check.to_csv(f"check/1860/{var}_check_1860.csv", index=False)

    else:
        parts = var.split("&")
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output"]
        variable_check = batch[output_var_list]
        if len(parts) == 2:
            left, right=parts
            variable_check['transcription_error'] =  None
            variable_check[f'correct_{left}'] = None
            variable_check[f'correct_{right}'] = None
        else:
            variable_check['transcription_error'] =  None
            variable_check[f'correct_{var}'] = None
        variable_check.to_csv(f"check/1860/{var}_check_1860.csv", index=False)


NameError: name 'var_list' is not defined